In [7]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime as dt

Data needed for configuration of the Option's pricings

In [ ]:
# Define the basket's tickers and weights
tickers = ['BHP.AX', 'CSL.AX', 'WDS.AX', 'MQG.AX']
weights = np.array([0.1, 0.35, 0.15, 0.4])

# Extracting closing prices as of Friday, 16 May 2025 from yfinance
data = yf.download(tickers, start='2025-05-16', end='2025-05-17')['Close']
closing_prices = data.loc['2025-05-16'].values

# Calculate weighted average (basket spot price, S0)
S0 = np.dot(weights, closing_prices)
print("Basket spot price S0 as of 16 May 2025:", S0)
print("Closing prices on 16 May 2025:", closing_prices)

#Download the latest Australian 10-year government bond yield for the risk-free rate, r, on 16 May 2025
rf = yf.Ticker("^AXY")
data = rf.history(end="2025-05-16") # Risk-free rate as of 16 May 2025
rf_rate = data["Close"].iloc[-1] / 100  # Convert from % to decimal
print(f"Risk-free rate as of 16 May 2025: {rf_rate:.4%}")
r = rf_rate

# Calculate daily log returns for volatility and correlation estimation
returns = np.log(data / data.shift(1)).dropna()

# Annualized volatilities
sigma = returns.std() * np.sqrt(252)
sigma = sigma.values

# Correlation matrix
corr_matrix = returns.corr().values

[*********************100%***********************]  1 of 4 completed

4 Failed downloads:
['CSL.AX', 'BHP.AX', 'WDS.AX', 'MQG.AX']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


KeyError: '2025-05-16'

In [ ]:
def binomial_european_call(S0, K, T, sigma, N):
    """Calculate European call option price using binomial model"""
    # Calculate time step
    dt = T/N
    print(f"dt: {dt}")
    
    # Calculate up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    print(f"u: {u}")
    d = 1/u
    print(f"d: {d}")
    
    # Calculate risk-neutral probability
    p = 1 / (u + 1)
    print(f"p: {p}")
    
    # Initialize stock price tree
    stock_tree = np.zeros((N+1, N+1))
    
    # Populate the stock price tree through time
    for i in range(N+1):
        for j in range(i+1):
            stock_tree[i, j] = S0 * (u ** j) * (d ** (i - j))
    
    # Initialize option value tree
    option_tree = np.zeros((N+1, N+1))
    
    # Calculate call option payoffs at expiration
    for j in range(N+1):
        option_tree[N, j] = max(0, stock_tree[N, j] - K)
    
    # Work backwards discounting and reflecting risk-neutral probabilities to calculate option value at each node
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            option_tree[i, j] = (p * option_tree[i+1, j+1] + (1-p) * option_tree[i+1, j])
    
    return option_tree[0, 0], stock_tree, option_tree

In [ ]:
K = 175
valuation_date = dt. datetime(2025, 5, 16)
expiry_date = dt.datetime(2025, 7, 17)
T = (expiry_date - valuation_date).days / 365
sigma = 0.2  # Replace with your basket's volatility estimate
N = 100

price, stock_tree, option_tree = binomial_european_call(S0, K, T, sigma, N)
print(f"European call basket option price: {price:.2f} AUD")
print("Time until expiry (T):", T)

In [ ]:
def monte_carlo_basket_call(S0, weights, K, T, rf_rate, sigma_vec, corr_matrix, n_paths=100):
    """
    Monte Carlo simulation for European basket call option.
    S0: array of spot prices for each asset
    weights: array of basket weights
    K: strike price
    T: time to maturity (in years)
    rf_rate: risk-free rate (annual, decimal)
    sigma_vec: array of volatilities for each asset
    corr_matrix: correlation matrix between assets
    n_paths: number of Monte Carlo paths
    """
    n_assets = len(S0)
    # Cholesky decomposition for correlated random numbers
    L = np.linalg.cholesky(corr_matrix)
    # Simulate correlated standard normals
    Z = np.random.normal(size=(n_paths, n_assets))
    correlated_Z = Z @ L.T
    # Simulate asset prices at maturity
    drift = (rf_rate - 0.5 * sigma_vec ** 2) * T
    diffusion = sigma_vec * np.sqrt(T)
    S_T = S0 * np.exp(drift + diffusion * correlated_Z)
    # Calculate basket value at maturity
    basket_T = S_T @ weights
    # Calculate payoff
    payoff = np.maximum(basket_T - K, 0)
    # Discounted expected payoff
    price = np.exp(-rf_rate * T) * np.mean(payoff)
    #Time step
    dt = T/N
    print(f"dt: {dt}")
    return price

In [15]:
K = 175
valuation_date = dt. datetime(2025, 5, 16)
expiry_date = dt.datetime(2025, 7, 17)
T = (expiry_date - valuation_date).days / 365
sigma = 0.2  # Replace with your basket's volatility estimate
N = 100

print("Annualized volatilities (sigma_vec):", sigma)
print("Correlation matrix (corr_matrix):\n", corr_matrix)
print("Time to maturity (T):", T)

Annualized volatilities (sigma_vec): 0.2


NameError: name 'corr_matrix' is not defined